# Tutorial: Running DFTB+ calculations from Jupyter

This tutorial demonstrates how on can run external DFTB+ calculations form Python script. 

This tutorial will also explain some ideas on the DFTB+ input


**Learning objectives:**

* to be able to run DFTB+ calculations from Jupyter
* to explain the key setup parameters


In [11]:
import os
import sys
import math
import copy

if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
    
from libra_py import units
from libra_py import QE_methods
from libra_py import normal_modes
import libra_py.DFTB_methods as dftb

import py3Dmol   # molecular visualization
import matplotlib.pyplot as plt   # plots
%matplotlib inline 

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}

colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

## 1. Prepare the DFTB+ input files

The only input files required for this tutorial are:

* **dftb_in.hsd** 
* **x1.gen** 
* **Slater-Koster files (elsewhere)**


### Description of the DFTB+ input file

The main input to the DFTB+ program is always called **dftb_in.hsd**. 

A full specification and further examples are given in the [DFTB+ manual](https://www.dftbplus.org/documentation/)

Here are some specifics for our example:

- the geometry is read from the "x1.gen" file 

```
Geometry = GenFormat {
   <<< "x1.gen"
}
```

- we compute MD using the Velocity verlet integrator
```
Driver = VelocityVerlet {
  ...
}	    
```

- we don't restrict motion of any atoms
```
MovedAtoms = 1:-1
```

- the dynamics is done withing the NVE ensemble, but initial velocities are
  taken from the Maxwell-Boltzmann distribution corresponding to a given temperature 
```
Thermostat = None {
  InitialTemperature [K] = 300
}
```

- as you initialize the velocities (randomly), it may happend that the total
  momentum of the system would not be zero, so the entire system would show 
  a translational (and rotational - due to non-zero total angular momentum)
  motion. But, we shall remove it - this is especially important for NA-MD
```
KeepStationary = Yes
```

- integration perameters: just 25 steps, 1 fs each
```
  Steps = 25
  TimeStep [fs] = 1.0
```  

- where to look for Slater-Koster files. This my local installation, your's would be different.
  You can get the files from [this website](https://www.dftb.org/parameters/download/).
  You also need to specify the maximal angular momentum of atomic orbitals on atoms of various type 
```
Hamiltonian = DFTB {
  SCC = Yes
  SlaterKosterFiles = Type2FileNames {
    Prefix = "/mnt/c/cygwin/home/Alexey-user/Soft/dftb/sk/3ob-3-1/"
    Separator = "-"
    Suffix = ".skf"
  }
  MaxAngularMomentum = {
    N = "p"
    O = "p"
    C = "p"
    H = "s"
  }
```
  


### Optional cleanup before the run

If you have already run the calculations in this folder, but would like to remove
all of the files DFTB+ and Libra generated in the previous run, you can use the following
auxiliary cleaning script

In [12]:
def clean():
    files_to_remove = ["autotest.tag", "band.out", "charges.bin",
                       "detailed.out", "dftb_pin.hsd",
                       "md.out", "md.xyz", "step0.gen", "steps0-1.gen"
                      ]
    
    for filename in files_to_remove:
        os.system(F"rm -rf {filename}")

clean()

## 2. Execute the DFTB+ calculations

This is basically to mimick the execution of the program from the command line.

You would have to specify the path to the corresponding executable. Since the input file is always called the same way, you don't need to provide that info in the call. 

On my system (laptop), I have to pre-compiled executables from [here](https://www.dftbplus.org/download/dftb-stable/). So, all I need to do is to select the one I want to use.

Just make sure that the present Jupyter file is located in the folder where you want the calculations to be run. This is also gonna be the place where all output files will be printed out.

In [13]:
EXE1 = "/mnt/c/cygwin/home/Alexey-user/Soft/dftb/dftbplus-18.2.x86_64-linux/bin/dftb+"
EXE2 = "/mnt/c/cygwin/home/Alexey-user/Soft/dftb/dftbplus-19.1.x86_64-linux/bin/dftb+"

os.system( "%s" % EXE2 )

0

## 3. Analysis and post-processing

Once the dynamics is completed, the file **md.xyz** is produced. 

We can utilize this file to:


### 3.1. Visualize the produced MD

In [14]:
f = open("md.xyz","r")
A = f.readlines()
f.close()

xyz = ""
for a in A:
    xyz = xyz + a
    

view = py3Dmol.view(width=800,height=400)  
view.setBackgroundColor('0xeeeeee')                                     
view.zoomTo()                                                           
view.addModelsAsFrames(xyz, 'xyz')
view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
view.animate({'reps':0, 'loop':'forward', 'step':5})
view.show()  

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### 3.2. Prepare a regular .gen file to be used in SCF single-point calculations

We can take any of the frames of the produced MD trajectory and prepare a .gen file with these coordinates. Who knows why you may want to do this, but it may be useful:

- to take one of the produces geometry as a starting point of a new MD

- to just compute properties (e.g. extra stuff like .cube files for cool orbitals) at some snapshots

- to be used withing more complicated workflows as in the step2 of this workflow

The syntax of the `xyz_traj2gen_sp` function is pretty straightforward:

- `md.xyz` - the name of the input file with MD
- `step0.gen` - the name of the generated output file
- `0` - this is the index of the snapshot to look at - the indexing starts with 0
- `C` - the option to control the header of the generated .gen files 

In [15]:
dftb.xyz_traj2gen_sp("md.xyz", "step0.gen", 0, "C")

### 3.3. Prepare a .gen with superimposed geometries 

This function is pretty cool - it generates a "supermolecule" by superimposing two geometries on top of each other. We can pick any timesteps along the available MD trajectory! This can be used in calculations of AO overlaps in two geometries - needed for time-derivative NACs calculations


The syntax of the `xyz_traj2gen_ovlp` function is pretty straightforward:

- `md.xyz` - the name of the input file with MD
- `steps0-1.gen` - the name of the generated output file
- `0` and `1` - are the indices of the two geometries to superimpose, the indexing starts from 0
- `C` - the option to control the header of the generated .gen files 

In [16]:
dftb.xyz_traj2gen_ovlp("md.xyz", "steps0-1.gen", 0, 1, "C")